In [229]:
import torch
import matplotlib.pyplot as plt
from torch import nn
import csv
import numpy as np
import torch.nn.functional as F

In [255]:
reader = csv.reader(open('data2-1.csv'), delimiter=',')
data = [float(row[1]) for row in reader]
# data = [float(number) for _, number in reader]
ratio = 0.8
train_datas = data[:int(len(data) * ratio)]
test_datas = data[int(len(data) * ratio):]
len(train_datas)

864

In [256]:
train_datas = torch.Tensor(train_datas)
test_datas = torch.Tensor(test_datas)

train_datas
train_datas.shape

torch.Size([864])

In [257]:
train_datas = F.normalize(train_datas,  dim=0)
test_datas = F.normalize(test_datas, dim=0)

In [258]:
train_data = []
test_data = []

for i in range(len(train_datas) // 50 - 1):
    train_data.append([train_datas[i*50:(i+1)*50], train_datas[(i+1)*50:(i+1)*50+10]])

for i in range(len(test_datas) // 10 - 1):
    test_data.append([test_datas[i*50:(i+1)*50], test_datas[(i+1)*50:(i+1)*50+10]])

In [259]:
train_data

[[tensor([0.0000, 0.0008, 0.0017, 0.0025, 0.0033, 0.0041, 0.0050, 0.0058, 0.0066,
          0.0074, 0.0082, 0.0090, 0.0099, 0.0107, 0.0115, 0.0123, 0.0131, 0.0139,
          0.0146, 0.0154, 0.0162, 0.0170, 0.0177, 0.0185, 0.0193, 0.0200, 0.0208,
          0.0215, 0.0222, 0.0230, 0.0237, 0.0244, 0.0251, 0.0258, 0.0265, 0.0272,
          0.0278, 0.0285, 0.0292, 0.0298, 0.0305, 0.0311, 0.0317, 0.0323, 0.0329,
          0.0335, 0.0341, 0.0347, 0.0352, 0.0358]),
  tensor([0.0363, 0.0368, 0.0373, 0.0378, 0.0383, 0.0388, 0.0393, 0.0397, 0.0402,
          0.0406])],
 [tensor([0.0363, 0.0368, 0.0373, 0.0378, 0.0383, 0.0388, 0.0393, 0.0397, 0.0402,
          0.0406, 0.0410, 0.0414, 0.0418, 0.0422, 0.0426, 0.0429, 0.0433, 0.0436,
          0.0439, 0.0442, 0.0445, 0.0448, 0.0451, 0.0453, 0.0455, 0.0458, 0.0460,
          0.0462, 0.0463, 0.0465, 0.0467, 0.0468, 0.0469, 0.0470, 0.0471, 0.0472,
          0.0473, 0.0473, 0.0474, 0.0474, 0.0474, 0.0474, 0.0474, 0.0473, 0.0473,
          0.0472, 0.0471,

In [260]:
# print(train_data.__class__.__name__)
# train_data = torch.from_numpy(train_data)
# test_data = torch.from_numpy(test_data)
# print(train_data.shape)
# print(np.max(train_data,axis=0).shape)

# plt.scatter(np.arange(len(train_data[0][0])), train_data[0][0])
# plt.scatter(np.array([len(train_data[0][0])]), train_data[0][1])

In [275]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(50,10)
        self.c1 = nn.Sigmoid()
        self.fc2 = nn.Linear(10,10)
        self.c2 = nn.Sigmoid()
        self.fc3 = nn.Linear(10,10)
        self.c3 = nn.Sigmoid()
        self.fc4 = nn.Linear(10,10)

    def forward(self, x):
        x = self.fc1(x)
        x = self.c1(x)
        x = self.fc2(x)
        x = self.c2(x)
        x = self.fc3(x)
        x = self.c3(x)
        x = self.fc4(x)
        return x

net = Net()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.8)
criterion = nn.CrossEntropyLoss()

def train(model, criterion, optimizer, epochs):
    for inputs, real_result in train_data:
        # print(inputs)
        inputs = torch.tensor(inputs, dtype=torch.float, requires_grad=True)
        real_result = torch.tensor(real_result, dtype=torch.float)
        for epoch in range(epochs):
            optimizer.zero_grad()
            output = model(inputs)
            # print(output,real_result)
            loss = criterion(output, real_result)
            # print(loss, output, real_result)
            loss.backward()
            optimizer.step()
    print("finished training")

def test(model):
    loss = 0.0
    for inputs, real_result in train_data:
        inputs = torch.tensor(inputs, dtype=torch.float)
        real_result = torch.tensor(real_result, dtype=torch.float)
        output = model(inputs)
        loss += criterion(output, real_result).item()
    print(f"test loss : {loss:.4f}")

In [277]:
train(net, criterion, optimizer, 100)
test(net)

C:\Users\Liu Xinyu\AppData\Local\Temp\ipykernel_22996\1659587870.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs, dtype=torch.float, requires_grad=True)
C:\Users\Liu Xinyu\AppData\Local\Temp\ipykernel_22996\1659587870.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  real_result = torch.tensor(real_result, dtype=torch.float)


finished training
test loss : 1.5731


C:\Users\Liu Xinyu\AppData\Local\Temp\ipykernel_22996\1659587870.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs, dtype=torch.float)
C:\Users\Liu Xinyu\AppData\Local\Temp\ipykernel_22996\1659587870.py:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  real_result = torch.tensor(real_result, dtype=torch.float)


In [102]:
torch.tensor(list([1,2,3]),dtype=torch.int)

tensor([1, 2, 3], dtype=torch.int32)

In [226]:
t1 = torch.Tensor(1)
t1

tensor([inf])

In [227]:
t2 = torch.rand(1)
t2

tensor([0.0098])

In [228]:
nn.MSELoss()(t1, t2)


tensor(inf)